This document describes the bid and response stratgies for various device types in TESS.

# Price Response

Bid/response strategies require information from the market about forward prices. The values are denoted, $P_{exp}$ and $P_{dev}$, the expected price and the expected standard deviation for the upcoming market interval. 

The forward market interval considered is based on the natural time-constant of the device.  When a device has a very fast response, it should consider fewer forward markets than when it's natural response is very slow.  A good rule of thumb is to consider forward price data for about 10 times the time-constant of the device.

In the absense of forward prices, the expected and variance are calculated from historical data using the last 24 hours of price data, i.e.,

$$
    P_{exp}(N) = \frac{1}{N} \sum_{t=-N}^{-1} P_t
$$

and

$$
    P_{dev}(N) = \frac{1}{N} \left( \sum_{t=-N}^{-1} [P_t-P_{exp}(N)]^2 \right)^{\frac12}
$$

where $N=288$ for a 24-hour history of 5-minute auctions.

# HVAC

For each occupancy mode, thermostatic devices depend on three user preference inputs, denoted $T_{des}$, $T_{min}$, and $T_{max}$, to convert temperatures to bid, and prices to bids. 

Table 1: HVAC User Preference Inputs (for each occupancy mode)

| Name          | Optional | Default     | Minimum | Maximum  | Definition
| ------------- | -------- | ----------- | ------- | -------- | ----------
| $T_{des}$     | No       | 72 degF     | 55 degF | 95 degF  | Desired indoor air temperature
| $T_{min}$     | No       | $T_{des}$-5 | 50 degF | 90 degf  | Minimum allowed temperature
| $T_{max}$     | No       | $T_{des}$+5 | 60 degF | 100 degf | Maximum allowed temperature

## HVAC Bid

In the absence of forward prices, an HVAC agent computes the bid price as follows

$$
    P_{bid} = P_{exp} - 3 m ~ P_{dev} ~ \frac{T_{obs}-T_{des}}{|T_{ref}-T_{des}|} 
$$

where $m\in\{-1,+1\}$ is the system mode ($-1$=cooling, $+1$=heating), $T_{obs}$ is the measured indoor air temperature.

When forward prices are given, an HVAC agent computes the bid prices as follows:

<font color=red>TODO: Claudio, describe method based on orderbook data.</font>

## HVAC Response

In the absence of forward prices, an HVAC agent computes a temperature setpoint as follows:

$$
    T_{set} = \left( T_{des} + \frac{P_{mkt}-P_{exp}}{3m~P_{dev}} |T_{des}-T_{ref}| \right)_{[T_{min},T_{max}]}
$$

where $P_{mkt}$ is the current price,

$$
    T_{ref} = \left\{ \begin{array}{lll}
        T_{min} & : & P \le P_{exp}
    \\
        T_{max} & : & P \gt P_{exp},
    \end{array} \right.
$$

and $x_{[a,b]}$ signifies that $x$ is restricted to the range $[a,b]$.

When forward prices are given, an HVAC agent computes the temperature setpoint as follows:

<font color=red>TODO: Claudio, describe method based on orderbook data.</font>

# Waterheater

Heat pump waters do not provide temperature readings, so it is relatively difficult to estimate the state-of-charge of the waterheater from the available data.  Consequently, the bidding strategy for the waterheater is based on a randomized bid.  

## Waterheater Bid
The bid price is computed as follows:

$$
    P_{bid} = P_{exp}+P_{dev} \left\{ \begin{array}{lll}
        -R\left(\sqrt{-2\ln(D_{exp})}\right) & : & D_{exp} \le 0.5
    \\
        R\left(\sqrt{-2\ln(1-D_{exp})}\right) & : & D_{exp} \le 0.5
    \end{array} \right.
$$

where the rational function 

$$
    R(\tau) = \frac{0.010328\tau^2+0.802853\tau+2.515517}{0.001308\tau^3+0.189269\tau^2+1.432788\tau+1}
$$

## Waterheater Response

The waterheater status is computed as follows:
$$
    status = \left\{ \begin{array}{lll}
        P_{clear} \le P_{bid} & : & on
    \\
        P_{clear} \gt P_{bid} & : & off
    \end{array} \right.
$$

<font color=red>TODO: Dave, check whether the relay interrupt changes the cumulative runtime</font>

# Photovoltaic

Rooftop PV always bids zero and always runs, regardless of the price.

# Battery

<font color=red>TODO: Dave, write-up the Behboodi V2G strategy</font>

# EV Charger

<font color=red>TODO: Dave, write-up the Behboodi V1G strategy</font>